# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some necessary and useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import collections
import numpy as np
import cv2
import math
%matplotlib inline


In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    """MP:
    https://www.quora.com/In-image-processing-applications-why-do-we-convert-from-RGB-to-Grayscale
    Vector vs Scalar: Image processing uses the concept of ‘comparing’ sections...Comparison in Grayscale 
    involves simple scalar algebraic operators (+ , -)....to differentiate colours, the methods are a bit more complex. 
    Usually, to get good results [when differentiating colors], some kind of Vector difference is needed. 
    This is computationally more complex, and still does not provide guaranteed better results. 
    Intensity data is usually sufficient...for edge detection."""
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

    """#MP:Perform canny BEFORE applying region of interest
    https://en.wikipedia.org/wiki/Canny_edge_detector
    
      The Process of Canny edge detection algorithm (5 steps)
    1.Apply Gaussian filter to smooth the image in order to remove the noise
    2.Find the intensity gradients of the image
    3.Apply non-maximum suppression to get rid of spurious response to edge detection
    4.Apply double threshold to determine potential edges
    5.Track edge by hysteresis: Finalize the detection of edges by suppressing all the other edges that are weak
    and not connected to strong edges."""

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    """
    MP:
    In the order of the pipeline, it is important to do the region of interest AFTER the canny. 
    If you reverse the order, the change to the black areas is detetected as an edge.
        
    https://en.wikipedia.org/wiki/Gaussian_blur
    In image processing, a Gaussian blur (also known as Gaussian smoothing) is the result of blurring an image
    by a Gaussian function. It is a widely used effect in graphics software, typically to reduce image noise and reduce
    detail,[edge detection filters are sensitive to noise]. ... Gaussian smoothing is also used as a pre-processing stage
    in computer vision algorithms in order to enhance image structures at different scales—see scale space representation
    and scale space implementation. """

def region_of_interest(img, vertices):
    
    """Applies an image mask. 
    Only keeps the region of the image defined by the polygon formed from `vertices`. 
    The rest of the image is set to black. """
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



def draw_lines(img, lines, color=[255, 0, 0], thickness=2): #MP NOTE: def name changed from draw_lines to draw_all_lines
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
     Returns an image with hough lines drawn.
    
    MP:
    http://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=houghlines#houghlines 
    The hough line transform is used to detect straight lines
    
    image – 8-bit, single-channel binary source image. The image may be modified by the function.
    rho – Distance resolution of the accumulator in pixels.
    theta – Angle resolution of the accumulator in radians.
    threshold – Accumulator threshold parameter. Only those lines are returned that get enough votes.
    minLineLength – Minimum line length. Line segments shorter than that are rejected.
    maxLineGap – Maximum allowed gap between points on the same line to link them.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


In [ ]:
#MP:  Set debug to false to allow debugging the algorithm for drawing onto the output images
debug=False


# We'll average information between frames
prev_lines = collections.deque([], 5)

def reset_prev_lines():
    prev_lines.clear()

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    MP:
    APPLY RECOMMENDED METHODS:
    1) Group lines into left and right arrays determined by their slopes
    2) Calculate the moving average of the slopes.
    3) Remove the lines which do not equal the average value.
    """
    lSlopeAvg=0
    rSlopeAvg=0
    slope_tolerance=.1
    #MP: Remove line values which are horizontal
    slope_tolerance_from_zero=.5
    bottom_y = img.shape[0]
    top_y = int(bottom_y /1.6)
    
    lLines = []
    rLines = []
    l = 1
    r = 1
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            if np.absolute(slope) == np.inf or np.absolute(slope) < slope_tolerance_from_zero:
                continue
            if debug:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
            #MP: If slope is less than zero, append to left line array
            if slope < 0:
                lSlopeAvg = lSlopeAvg + (slope - lSlopeAvg) / l
                l += 1
                if np.absolute(lSlopeAvg - slope) < slope_tolerance :
                    lLines.append((x1,y1))
                    lLines.append((x2,y2))
            else:
                #MP: If slope is not less than zero, append to right line array
                rSlopeAvg = rSlopeAvg + (slope - rSlopeAvg) / r
                r += 1
                if np.absolute(rSlopeAvg - slope) < slope_tolerance :
                    rLines.append((x1,y1))
                    rLines.append((x2,y2))               
    
    """
    MP:
    1) After grouping the L and R lines, apply cv2.fitline which fits the sets of points to a single line.
    2) cv2.fitline: Output line parameters. It should be a vector of 4 elements (like Vec4f) - (vx, vy, x0, y0), 
    where (vx, vy) is a normalized vector collinear to the line and (x0, y0) is a point on the line. 
    """
    if len(lLines) > 0 and len(rLines) > 0  :
        [left_vx,left_vy,left_x,left_y] = cv2.fitLine(np.array(lLines, dtype=np.int32), cv2.DIST_L2,0,0.01,0.01)      
        left_slope = left_vy / left_vx
        left_b = left_y - (left_slope*left_x)

        [right_vx,right_vy,right_x,right_y] = cv2.fitLine(np.array(rLines, dtype=np.int32), cv2.DIST_L2,0,0.01,0.01)    
        right_slope = right_vy / right_vx
        right_b = right_y - (right_slope*right_x)

        #MP:  Calculate the average of this line, and that of previous frames    
        prev_lines.append((left_b, left_slope, right_b, right_slope))
    
    if len(prev_lines) > 0: 
        avg = np.sum(prev_lines, -3) /len(prev_lines)
        left_b = avg[0]
        left_slope = avg[1]
        right_b = avg[2]
        right_slope = avg[3]

        """
        Using the slope and the y-inercept: 
        Calculate the x coordinates of the start and end points at the top of the road and at the bottom of the road.      
        """
        
        ltop_x = (top_y - left_b) / left_slope
        lbottom_x = (bottom_y - left_b) / left_slope

        rtop_x = (top_y - right_b) / right_slope
        rbottom_x = (bottom_y - right_b) / right_slope

        cv2.line(img, (lbottom_x, bottom_y), (ltop_x, top_y), color, thickness)
        cv2.line(img, (rbottom_x, bottom_y), (rtop_x, top_y), color, thickness)



## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
#Import necessary libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import collections
import numpy as np
import cv2
import math
%matplotlib inline
import os

folder = "test_images/"
images = os.listdir(folder)

#MP REGION OF INTEREST (ROI)
def get_roi(w, h):
    v1 = (int(w*.11),int(h))
    v2 = (int(w*.44),int(h*.60))
    v3 = (int(w*.57),int(h*.60))
    v4 = (int(w*.97),int(h))
    return([[v1,v2,v3,v4]])


canny_l_threshold = 70
canny_h_threshold = 200

rho=1
theta=np.pi/180
threshold=20
min_line_len=10
max_line_gap=20

frame=1

def pipeline(image):
    # Identify region of interest
    roi_vertices = np.array(get_roi(image.shape[1],image.shape[0]), dtype=np.int32)  
    
    # Add extra blur -- MP Changed kernel size from 7 to 3 
    blurred = gaussian_blur(image, 3)
    
    # Run Canny for edge detection
    edges = canny(blurred, canny_l_threshold, canny_h_threshold)   

    # Focus on the region of interest
    roi = region_of_interest(edges, roi_vertices)
    
    """MP: After we have the Canny edges,
    then run Hough transform
    to try to find the lines"""
    
    lines = hough_lines(roi, rho, theta, threshold, min_line_len, max_line_gap)
               
    # Overlay the Hough lines we calculated onto the original image
    
    if debug: 
        return region_of_interest(weighted_img(lines, image),roi_vertices)
    else:
        return weighted_img(lines, image)
       

for imageFile in images:
    original_filename, ext = os.path.splitext(imageFile)
    if ext != ".jpg":
        continue
        
    image = mpimg.imread(folder + imageFile)
    reset_prev_lines()
    output = pipeline(image)
    
    plt.figure()
    plt.imshow(output, cmap='Greys_r')
    
    new_filename = os.path.join(folder+ "out/", original_filename + "_out" + ext)
    mpimg.imsave(new_filename, output, cmap='Greys_r')
    
    


run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import os
os.listdir()
from moviepy.editor import VideoFileClip
from IPython.display import HTML



In [ ]:
def process_image(image):
    """ NOTE: The output you return should be a color image (3 channel) for processing video below
    TODO: put your pipeline here,
    you should return the final output (image with lines are drawn on lanes)"""
    
    result = pipeline(image)
    
    return result


Let's try the one with the solid white lane on the right first ...

In [ ]:
#Lets try with the solid white lane

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4",audio=False) #MP added " ,audio=False
reset_prev_lines()
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
#PLAY THE VIDEO INLINE USING HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
#NOW, GENERATE THE SOLID YELLOW LINE VIDEO:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip("solidYellowLeft.mp4",audio=False)   #MP added " ,audio=False
reset_prev_lines()
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


In [ ]:
"""
NEW REFLECTION SUMMARY:

SUMMARY OF MY PIPELINE:
1) Identify image shape (roi_vertices)
2) Add (gaussian) blur: I decreased from 7 to 3 kernel size to improve Canny edge detection.
3) Run canny for edge detection
4) Focus on the region of interest
5) Run Hough transform to try to find the lines
6) Draw lines

WEAKNESSES OF MY PIPELINE:
Weaknesses of my pipeline would be problems caused by:
-Changes in the field of view: Obstructions such as other cars in the field of view
-Changes in the linearity of lane lines: Extreme curved lines on turns
-Changes in light conditions: Extreme low light at night
-Changes in light reflection: Reflections and low contrast from water on road
-Missing lines and sections of lines: Due to snow and puddles on road.

WHAT I WOULD IMPROVE ON IN ADVANCED LANE LINES: COLOR THRESHHOLDING
What I would like to add for the Advanced Lane Lines, to correct the curved line problem:
In my Optional Challenge saved video, my right line jumps for a sec, due to curvature of line. 
For curving lines, color threshholding would help delineate the lines.

------------------------------

NEW REFLECTION: MY PIPELINE
Using our 1st static test image, we display the image dimensions of 540 x 960 and 3 color channels.

GRAYSCALE:
One of the first things we do is convert the 3 channel RGB to 1 channel (grayscale).
That will help the Canny transform, which looks at the gradient contrast. 

GAUSSIAN BLUR:
We then apply the Gaussian blur, because the blurred average effect helps us to only focus on strongly contrasting boundaries (not noisy weak boundaries). The parameters are image and kernel size. Kernel size is the size of the region that is averaged.  I originally used a kernel size of 7, which is probably on the higher end of the best range, so I modified the value down to 3.

CANNY TRANSFORM:
After we blur, we find the edges, using Canny. The parameters take the blurred gray scale image, and create a binary image determined by the strength of the pixelsBetween the low and high threshhold, the value is one. Otherwise, the value is 0. I used a low threshhold of 70 and a high threshhold of 200.

ROI:
To identify the region of interest roi, we perform a crop of the edges image, we use imshape, to pull the height and width values of the image, and create 4 vertices: v1,v2,v3,v4. They values of the vertices are in percentages:
def get_roi(w, h):
    v1 = (int(w*.11),int(h)), 
    v2 = (int(w*.44),int(h*.60))
    v3 = (int(w*.57),int(h*.60))
    v4 = (int(w*.97),int(h))
    return([[v1,v2,v3,v4]])
My calculated vertices are: (105, 540) (422, 324) (547, 324) (931, 540)

HOUGH LINES TRANSFORM:
Hough lines is doing the line calculations, and then calling the draw function.

Hough_lines takes 6 arguments:  
rho=1 (maximum resolution in pixels)
theta=np.pi/180 (1 degree)
threshold=20 (Number of votes to be considered a binary point)
min_line_len=10 (Minimum length of line)
max_line_gap=20 (I do not want gap bigger than 20)

In hough lines transform, we can transform a line to a point, and then transform that point back to a line: 
The problem with calculating y = mx + c is that m, or slope of a vertical line, is undefined, 
Every y has the same value of x. Delta y over delta x is undefined, because the x denominator is zero.

So instead of using the simple equation of a line, we us:  rho = x cos theta + y sin theta, where rho is the perpendicular distance from the origin to the line, and theta is the angle formed by this perpendicular line and horizontal axis measured counterclockwise.  (Rho is rows, theta is columns.)

We are identifying theta values that correspond to rho values, and we are saving those pairs.
We find where there is the maximum amount of overlap in the lines, and that corresponds to a strong binary point.

Threshhold is the number of votes required in our grid to count and even be considered a line.
Min_line_length is obviously the minimum length of what is considered a line
Max_line_gap is the maximum gap allowed between lines.

WEIGHTED IMAGE:
Weighted_img is to overlay the original image onto our lane line markers image.

Weighted_img has 5 arguments:
lines: The lane lines from the Hough transform
img: The original image
alpha ?=0.8, if it is less than one, the lines image will be just a bit see-through (tranluscent)
beta ?=1.0, it is one, so the original image is not transparent at all
lambda ?=0.0, we set to zero, which means no diffusion.

VIDEO PROCESSING:
So our pipeline now looks like this:
def pipeline(image):
    # Identify region of interest
    roi_vertices = np.array(get_roi(image.shape[1],image.shape[0]), dtype=np.int32)  
    
    # Add extra blur -- MP Changed kernel size from 7 to 3 
    blurred = gaussian_blur(image, 3)
    
    edges = canny(blurred, canny_l_threshold, canny_h_threshold)   

    roi = region_of_interest(edges, roi_vertices)
   
    lines = hough_lines(roi, rho, theta, threshold, min_line_len, max_line_gap)
               
    # Overlay the Hough lines we calculated onto the original image
    if debug: 
        return region_of_interest(weighted_img(lines, image),roi_vertices)
    else:
        return weighted_img(lines, image)

APPLY THE PIPELINE FRAME-BY-FRAME:
   white_output = 'white.mp4'
   clip1 = VideoFileClip("solidWhiteRight.mp4",audio=False)
   reset_prev_lines()
   white_clip = clip1.fl_image(process_image)
   %time white_clip.write_videofile(white_output, audio=False)

DISPLAY THE VIDEO INLINE:
   HTML("""
   <video width="960" height="540" controls>
       <source src="{0}">
   </video>
   """.format(white_output))

DRAW LINES:
Draw_lines is creating the line segments x1, y1 and x2, y2 for the left and right line segments, and near the end, it will create only one long x1,y1 and x2, y2 pair for left and right

Hough lines is generating a list of start points and end points, and it hands it off to draw_lines, which connects the points using a line on a blank template. But we do NOT want a series of small line segments, we only want TWO lines, one final left, and one final right.

So the draw_lines algorithm uses slopes and centers to calculate the coordinates of the lines.

APPLY RECOMMENDED METHODS:
    1) Group lines into left and right arrays determined by their slopes
    2) Calculate the moving average of the slopes.
    3) Remove the lines which do not equal the average value.
    lLines = []
    rLines = []
    lSlopeAvg=0
    rSlopeAvg=0

    4) After grouping the L and R lines, apply cv2.fitline 
    which fits the sets of points to a single  line.

    5) cv2.fitline: Output line parameters. 
    It should be a vector of 4 elements (like Vec4f) - (vx, vy, x0, y0), 
    where (vx, vy) is a normalized vector collinear to the line and (x0, y0) is a point on the line. 

    6) Using the slope and the y-inercept: 
    Calculate the x coordinates of the start and end points at the top of the road and at the
    bottom of the road.  

"""

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  
Does it still work?  YES, IT WORKS FAIRLY WELL.  IT IS OVER-EXTENDING THE CURVE JUST A BIT. 

Can you figure out a way to make it more robust?  
I WOULD LIKE TO FIGURE OUT HOW TO HAVE IT FIT THE CURVE WITHOUT OVER-EXTENDING.

If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
#NOW, GENERATE THE CHALLENGE EXTRA VIDEO:
challengeExtra_output = 'challengeExtra.mp4'
clip3 = VideoFileClip('challenge.mp4', audio=False)  #MP added " ,audio=False
reset_prev_lines()
challengeExtra_clip = clip3.fl_image(process_image)
%time challengeExtra_clip.write_videofile(challengeExtra_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challengeExtra_output))